# How to call functions with ChatGPT

This notebook covers how to use the Chat Completions API (aka ChatCPT) in combination with external functions to extend the capabilities of GPT models.

`Function calling` is an optional capabilities in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications.

The `Request-ChatGPT` or `Request-ChatCompletion` commands has optional parameters for use of function call. You can provide function specifications to GPT models.

## Overview
This notebook contains the following 2 sections:

How to generate function arguments: Specify a set of functions and use the API to generate function arguments.  
How to invoke functions with model generated arguments: Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

### Create function specifications

In order to have the GPT model perform a function call, a function specification has to be created and given as a parameter to the model.

In [1]:
# imports
Import-Module ..\PSOpenAI.psd1 -Force

#### Method 1: Defines function specifications manually

You can create hashtable that represents function specification manually.

This example creates a specifications for a `Get-WeatherByCityAndDate` function that takes two arguments: a `city` name and a `date`.

In [2]:
$GetWeatherFunction = @{
    name = "Get-WeatherByCityAndDate"                            # The name of function
    description = "Get specific date of weather in the city"    # Brief description of the function
    parameters = @{
        type = "object"
        properties = @{
            "city" = @{                                         # The name of first argument
                type = "string"                                 # Type of the argument
                description = 'The city name, e.g. New York'    # Brief description (optional)
            }
            "date" = @{                                         # The name of second argument
                type = "string"
                format = "date"
            }
        }
    required = ("city", "date")                                 # Arguments that should be specified
    }
}

#### Method 2: Generate function specifications from PowerShell commands

PSOpenAI has a `New-ChatCompletionFunction` that helps generating specifications from PowerShell command definitions.

This example generates specification of the `Test-Connection` command with 3 parameters.

In [3]:
$PingFunction = New-ChatCompletionFunction -Command "Test-Connection" -IncludeParameters ('TargetName', 'Count', 'Delay')

`New-ChatCompletionFunction` can convert any PowerShell command into a function specification for ChatGPT, but the design of most PowerShell commands is not optimized for passing to the GPT model.

It is best to create your own properly designed PowerShell functions for optimal results.

In [4]:
# This function is an example of a PowerShell function optimized for the GPT model.
<#
Tips: 
    1. Functions should be named appropriately.
    2. The name of functions should be less than 64 characters and may only contain alphanumeric, hyphens, and underscores.  
    3. Always write comment-based help for function.
    4. Be sure to write a description of the function itself and each parameter.
#>
function Get-WeatherByCityAndDate {
    <#
        .DESCRIPTION
        This command returns specific date of weather in the city.

        .PARAMETER City
        Specifies the city name, e.g. New York

        .PARAMETER Date
        Specifies the date to get information.

        .PARAMETER Format
        The temperature unit to use. Default is "celsius".
    #>
    param
    (
        <#
        Tips: 
          1. Parameters should be of the appropriate type.
          2. Use only primitive types for parameters. Avoid .NET classes.
          3. Required parameters use the Mandatory attribute.
          4. Use Validation attributes if there are restrictions on the values that can be specified.
        #>
        [Parameter(Mandatory = $true)]
        [string]$City,

        [Parameter(Mandatory = $true)]
        [datetime]$Date,

        [Parameter()]
        [ValidateSet('celsius', 'fahrenheit')]
        [string]$Format = 'celsius'
    )
    
    <#
    Tips:
      1. Interactive commands with confirmation dialogs or input by users are not good.
      2. The object to be returned by the function is best suited as [string], [HashTable], or [PSCustomObject].
      3. Select only the necessary information to be returned. Outputs excessive information is not a good idea.
    #>

    # In reality, the information is obtained from API or Database.
    $result = @{
        city                = $city
        date                = $date
        weather             = 'Sunny'
        highest_temperature = 30.0
        lowest_temperature  = 21.2
        temperature_format  = $Format
    }

    Write-Output $result
}

### Instructs the GPT model to create arguments for function calls.

Give the function specifications in the `-Functions` parameter of the `Request-ChatCompletion`.  
If the GPT model chooses to make a function call from the content of the message, the name and arguments of the function are stored in `functions_call` of the response message.

In [5]:
$Message = 'What is the weather in Philadelphia on June 1, 2023?'
$GPTAnswer = Request-ChatCompletion -Message $Message -Model gpt-3.5-turbo-0613 -Functions $GetWeatherFunction
$GPTAnswer.choices[0].message | Format-List


role          : assistant
content       : 
function_call : @{name=Get-WeatherByCityAndDate; arguments={
                  "city": "Philadelphia",
                  "date": "2023-06-01"
                }}




In this example, the GPT model has selected the appropriate calling function and created arguments from the natural language message, but it does not actually execute the `Get-WeatherByCityAndDate` function. It is up to you to decide whether and how to execute the function.

To give the result of a function execution to ChatGPT and get an answer for the user, do the following.

In [7]:
# Invoke functions explicitly by yourself
$FunctionResult = Get-WeatherByCityAndDate -City "Philadelphia" -Date "2023-06-01"
Write-Host 'Result of execution the function:'
$FunctionResult

# To give the result of a function execution to ChatGPT and get an answer
$FunctionResultAsJson = $FunctionResult | ConvertTo-Json
$GPTAnswer2nd = $GPTAnswer | Request-ChatCompletion -Message $FunctionResultAsJson -Role 'function' -Name 'Get-WeatherByCityAndDate' -Model gpt-3.5-turbo-0613
Write-Host 'Answer by ChatGPT:'
$GPTAnswer2nd.choices[0].message | Format-List

Result of execution the function:

Name                           Value
----                           -----
temperature_format             celsius
lowest_temperature             21.2
city                           Philadelphia
date                           2023/06/01 0:00:00
weather                        Sunny
highest_temperature            30


Answer by ChatGPT:


role    : assistant
content : On June 1, 2023, the weather in Philadelphia is expected to be sunny. The lowest temperat
          ure will be around 21.2ºC (70.2ºF), and the highest temperature is forecasted to be aroun
          d 30ºC (86ºF).




## How to invoke functions with model generated arguments

Specifying `Auto` for `-InvokeFunctionOnCallMode` parameter in `Request-ChatCompletion`, it will implicitly execute the function and return the result when the GPT model requests function call.

Note: Only PowerShell commands can be executed. Native commands or commands that require another runtime such as Python cannot be executed and will generate an error.

In [8]:
$Message = 'Ping the Google Public DNS address three times and briefly report the results.'
$GPTPingAnswer = Request-ChatCompletion -Message $Message -Model gpt-3.5-turbo-0613 -Functions $PingFunction -InvokeFunctionOnCallMode Auto
$GPTPingAnswer.choices[0].message | Format-List


role    : assistant
content : I have pinged the Google Public DNS address three times. Here are the results:
          
          Ping 1: 8.8.8.8 - Success, Latency: 8ms
          Ping 2: 8.8.8.8 - Success, Latency: 8ms
          Ping 3: 8.8.8.8 - Success, Latency: 7ms




The contents of implicitly executed commands are stored in the message history.

In [9]:
[pscustomobject]($GPTPingAnswer.History | ? {$_.keys -contains 'function_call'}) | select -ExpandProperty function_call | Format-List


name      : Test-Connection
arguments : {
              "TargetName": ["8.8.8.8"],
              "Count": 3
            }


